In [2]:
import pandas as pd
import numpy as np
from mysql.connector import connect
import json
from os import path
from utils import get_db_creds
import sqlalchemy

In [3]:
db_creds = get_db_creds()
connection = connect(**db_creds)

In [4]:
conn = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'
            .format(db_creds["user"], db_creds["password"], db_creds["host"], db_creds["database"])).connect()

In [5]:
users_df = pd.read_csv("users.csv")
users_df = users_df.rename(columns={"hashed_password": "password"})
users_df["username"] = users_df["username"].str.replace(" ", "_")
username_set = set()
for i in range(len(users_df)):
    counter = 1
    while (users_df.iloc[i]["username"] in username_set):
        users_df.loc[users_df.index == i, "username"] = users_df.iloc[i]["username"] + "_" + str(counter)
        counter += 1
    username_set.add(users_df.iloc[i]["username"])
users_df.head()

,user_id,username,password
0,1,Elizebeth_Marshall,password
1,2,Dwayne_Stpierre,password
2,3,Kim_Mclaughlin,password
3,4,Frank_Barrette,password
4,5,Timothy_Hackman,password


In [6]:
users_df.to_sql(name="User", con=conn, if_exists="append", index=False,)

In [7]:
cuisines_df = pd.read_csv("cuisines.csv")
cuisines_df.head()

,cuisine_id,cuisine_name
0,1,north-american
1,2,global
2,3,asian
3,4,german
4,5,italian


In [8]:
cuisines_df.to_sql(name="Cuisine", con=conn, if_exists="append", index=False,)

In [22]:
tags_df = pd.read_csv("tags.csv")
tags_df = tags_df.fillna("")
tags_df.head()

,tag_text,tag_id
0,,1
1,broccoli,2
2,course,3
3,pork-sausage,4
4,salads,5


In [23]:
tags_df.to_sql(name="Tag", con=conn, if_exists="append", index=False,)

In [11]:
ingr_map_df = pd.read_csv("ingr_map.csv")
# ingr_map_df = ingr_map_df.rename(columns={"replaced": "ingredient_name", "id": "ingredient_id"})
# ingr_map_df = ingr_map_df[["ingredient_name", "ingredient_id"]]
# ingr_map_df = ingr_map_df.drop_duplicates()
# ingr_map_df.loc[ingr_map_df["ingredient_id"] == 0, "ingredient_id"] = ingr_map_df["ingredient_id"].max() + 1
ingr_map_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8023 entries, 0 to 11658
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  8023 non-null   object
 1   ingredient_id    8023 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 188.0+ KB


In [12]:
ingr_map_df.to_sql(name="Ingredient", con=conn, if_exists="append", index=False,)

In [13]:
recipe_df = pd.read_csv("recipe_155k_with_all_updated_columns.csv")
columns_needed = ["recipe_id", "creator_id", "recipe_name", "serves",
                 "date_submitted", "cuisine_id", "description",
                 "recipe_text", "calories", "time_to_prepare",
                 "img_url"]
recipe_df = recipe_df[columns_needed]
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155751 entries, 0 to 155750
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   recipe_id        155751 non-null  int64  
 1   creator_id       155751 non-null  int64  
 2   recipe_name      155751 non-null  object 
 3   serves           155751 non-null  int64  
 4   date_submitted   155751 non-null  object 
 5   cuisine_id       155751 non-null  int64  
 6   description      152310 non-null  object 
 7   recipe_text      155751 non-null  object 
 8   calories         155751 non-null  float64
 9   time_to_prepare  155751 non-null  int64  
 10  img_url          155751 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 13.1+ MB


In [14]:
recipe_df.to_sql(name="Recipe", con=conn, if_exists="append", index=False,)

In [15]:
recipe_ingredients_df = pd.read_csv("recipe_ingredients_df.csv")
recipe_ingredients_df = recipe_ingredients_df.drop_duplicates(subset=["recipe_id", "ingredient_id"])
recipe_ingredients_df.loc[recipe_ingredients_df["ingredient_id"] == 0, "ingredient_id"] = ingr_map_df["ingredient_id"].max()
recipe_ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1364897 entries, 0 to 1366603
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   recipe_id         1364897 non-null  int64  
 1   ingredient_id     1364897 non-null  int64  
 2   quantity          1279760 non-null  float64
 3   measurement_unit  1279760 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 52.1+ MB


In [16]:
recipe_ingredients_df.to_sql(name="RecipeIngredient", con=conn, if_exists="append", index=False,)

In [17]:
interactions_df = pd.read_csv("interactions_df.csv")
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174305 entries, 0 to 174304
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           174305 non-null  int64 
 1   recipe_id         174305 non-null  int64 
 2   interaction_date  174305 non-null  object
 3   rating            174305 non-null  int64 
 4   review            174288 non-null  object
dtypes: int64(3), object(2)
memory usage: 6.6+ MB


In [18]:
interactions_df.drop_duplicates(subset=["user_id", "recipe_id"]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172808 entries, 0 to 174304
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           172808 non-null  int64 
 1   recipe_id         172808 non-null  int64 
 2   interaction_date  172808 non-null  object
 3   rating            172808 non-null  int64 
 4   review            172791 non-null  object
dtypes: int64(3), object(2)
memory usage: 7.9+ MB


In [19]:
interactions_df = interactions_df.drop_duplicates(subset=["user_id", "recipe_id"])

In [20]:
interactions_df.to_sql(name="Interaction", con=conn, if_exists="append", index=False,)

In [24]:
recipe_tags_df = pd.read_csv("recipe_tags.csv")
recipe_tags_df.head()

,recipe_id,tag_id
0,137739,257
1,137739,166
2,137739,321
3,137739,242
4,137739,514


In [25]:
recipe_tags_df.to_sql(name="RecipeTag", con=conn, if_exists="append", index=False,)

In [ ]:
# KEEP iloc[292740] for recipe_ingredients_df in mind
recipe_ingredients_df.iloc[292740]

In [ ]:
ingr_map_df[ingr_map_df["ingredient_id"] == 0]